# MODELO PREDICTIVO - ANÁLISIS DE ÉXITO

In [4]:
# CARGA DE DATASET Y LIBRERÍAS
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer


df = pd.read_csv('/content/sample_data/Datos_abiertos_cepre.csv')
df

,IDHASH,COLEGIO,COLEGIO_DEPA,COLEGIO_PROV,COLEGIO_DIST,COLEGIO_PAIS,COLEGIO_ANIO_EGRESO,ESPECIALIDAD,ANIO_POSTULA,CICLO_POSTULA,...,ANIO_NACIMIENTO,NACIMIENTO_PAIS,NACIMIENTO_DEPA,NACIMIENTO_PROV,NACIMIENTO_DIST,SEXO,CALIF_FINAL,INGRESO,MODO_INGRESO,FECHA_CORTE
0,54CB3EB25FC93C1FAC691E2351D8E561EA7A5C1CC0C840...,Corazón de Jesús,CALLAO,CALLAO,CALLAO,PERU,2013,SIN ESPECIALIDAD,2016,2,...,1997,Peru,CALLAO,CALLAO,CALLAO,MASCULINO,4.491,NO,SIN INGRESO,20240529
1,5A18BE74478BB8BAA3D1E1B0830AF43D883E55471BEC50...,Aurelio miro quesada sosa,LIMA,LIMA,LA MOLINA,PERU,2013,SIN ESPECIALIDAD,2016,2,...,1997,Peru,LIMA,LIMA,LIMA,FEMENINO,2.045,NO,SIN INGRESO,20240529
2,5BF8EBFFA46ED932AE6135BD683BB792AC8E4B9C068DAD...,Daniel Alomia Robles,LIMA,LIMA,SAN JUAN DE LURIGANCHO,PERU,2015,SIN ESPECIALIDAD,2016,2,...,2000,Peru,LIMA,LIMA,INDEPENDENCIA,MASCULINO,0.642,NO,SIN INGRESO,20240529
3,3CCEB93417E4F004B107B6389797B7948BA7F051FCB405...,alfonso ugarte,LIMA,LIMA,LURIGANCHO,PERU,2015,SIN ESPECIALIDAD,2016,2,...,1999,Peru,LIMA,LIMA,SAN JUAN DE LURIGANCHO,FEMENINO,0.535,NO,SIN INGRESO,20240529
4,909D65D35B90A659D366B3F8B2C467594E550452ACBA6C...,pamer,LIMA,LIMA,PUENTE PIEDRA,PERU,2015,SIN ESPECIALIDAD,2016,2,...,1999,Peru,LIMA,LIMA,PUENTE PIEDRA,MASCULINO,0.591,NO,SIN INGRESO,20240529
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33804,B050E6AED4453C8BF27BB8DF02AC1598765049912CE503...,Saco Oliveros Apeiron,LIMA,LIMA,SAN MARTIN DE PORRES,PERU,2022,MATEMÁTICA,2023,2,...,2005,Venezuela,SIN DATO,SIN DATO,SIN DATO,MASCULINO,11.876,SI,DIRECTO,20240529
33805,CE557A05737E8E4292F5256671F29429D2B8A5F05B85BA...,Saco Oliveros,LIMA,LIMA,SAN JUAN DE MIRAFLORES,PERU,2021,INGENIERIA AMBIENTAL,2023,2,...,2005,Peru,LIMA,LIMA,SAN JUAN DE MIRAFLORES,FEMENINO,11.574,SI,DIRECTO,20240529
33806,87012F48EA0EE2FF730075E93A8C99916E5F6FC22A0BD4...,Horacio Zeballos Gamez 7227,LIMA,LIMA,LIMA,PERU,2022,SIN ESPECIALIDAD,2023,2,...,2004,España,SIN DATO,SIN DATO,SIN DATO,MASCULINO,2.759,NO,SIN INGRESO,20240529
33807,A39C399FAF45C7970213A5D758D5E50F8D8859B894F095...,Saco Oliveros,LIMA,LIMA,COMAS,PERU,2022,SIN ESPECIALIDAD,2023,2,...,2006,Peru,LIMA,LIMA,LIMA,MASCULINO,5.650,NO,SIN INGRESO,20240529


In [5]:
# AGREGAR COLUMNA 'CICLO' PARA CONSIDERAR EL DATO EN UNA SOLA COLUMNA
df['CICLO'] = df[['ANIO_POSTULA', 'CICLO_POSTULA']].astype(str).agg('-'.join, axis=1)
df['CICLO']

,CICLO
0,2016-2
1,2016-2
2,2016-2
3,2016-2
4,2016-2
...,...
33804,2023-2
33805,2023-2
33806,2023-2
33807,2023-2


In [6]:
# AGREGAR COLUMNA 'EDAD' RESTANDO EL AÑO DE POSTULACIÓN Y AÑO DE NACIMIENTO
df['EDAD'] = df['ANIO_POSTULA'] - df['ANIO_NACIMIENTO']
df['EDAD']

,EDAD
0,19
1,19
2,16
3,17
4,17
...,...
33804,18
33805,18
33806,19
33807,17


In [7]:
# UN PARÁMETRO A CONSIDERAR EN EL ANÁLISIS Y MODELO PREDICTIVO ES LA 'PREPARACION' QUE LOS ALUMNOS CON DESEOS DE INGRESAR A LA UNI DEBEN CONSIDERAR PARA CUMPLIR TAL OBJETIVO
# GENERALMENTE, AL EGRESAR DEL COLEGIO, LOS ALUMNOS OPTAN POR MATRICULARSE EN ACADEMIAS PRE UNIVERSITARIAS ASÍ COMO EN LA CEPRE-UNI, POR LO QUE SE ASUME PARA EL ANÁLISIS QUE A MAYOR PREPARACIÓN, MÁS POSIBILIDADES DEL ALUMNA DE INGRESAR A LA UNI
# AGREGAR COLUMNA 'PREPARACION' RESTANDO AÑO DE POSTULACIÓN Y AÑO DE EGRESO DEL COLEGIO, EXISTEN UN PAR DE CASOS EN LOS QUE ALUMNOS QUE AÚN NO HABÍAN EGRESADO DEL COLEGIO, SE MATRICULARON EN LA CEPRE-UNI, POR LO QUE ESTE NO GENERARÁ INCONSISTENCIA EN LOS DATOS
df['PREPARACION'] = df['ANIO_POSTULA'] - df['COLEGIO_ANIO_EGRESO']
df['PREPARACION']

,PREPARACION
0,3
1,3
2,1
3,1
4,1
...,...
33804,1
33805,2
33806,1
33807,1


In [8]:
# AGREGAR COLUMNA 'COLEGIO_DEPA_PROV', SE CONSIDERA LA CONCATENACIÓN PARA DE AMBAS COLUMNAS PARA LUEGO AGRUPAR LOS DATOS POR PROVINCIAS-DEPARTAMENTOS Y DIFERENCIARLOS ADEMÁS DE LAS PROVINCIAS METROPOLINADA DE LIMA Y LA PROVINCIA CONSTITUCIONAL DEL CALLAO, DONDE EN ESTOS ÚLTIMOS SE TIENE MAYOR CANTIDAD DE REGISTROS
df['COLEGIO_DEPA_PROV'] = df[['COLEGIO_DEPA', 'COLEGIO_PROV']].astype(str).agg('-'.join, axis=1)
df['COLEGIO_DEPA_PROV']

,COLEGIO_DEPA_PROV
0,CALLAO-CALLAO
1,LIMA-LIMA
2,LIMA-LIMA
3,LIMA-LIMA
4,LIMA-LIMA
...,...
33804,LIMA-LIMA
33805,LIMA-LIMA
33806,LIMA-LIMA
33807,LIMA-LIMA


In [9]:
# REASIGNAMOS EL NOMBRE DE LA COLUMNA 'IDHASH' POR EL DE 'ALUMNOS' Y VALIDAMOS LA INFORMACIÓN DEL DF CON LA ACTUALIZACIÓN HASTA EL MOMENTO
df.rename(columns={'IDHASH': 'ALUMNOS'}, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33809 entries, 0 to 33808
Data columns (total 27 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ALUMNOS              33809 non-null  object 
 1   COLEGIO              33809 non-null  object 
 2   COLEGIO_DEPA         33809 non-null  object 
 3   COLEGIO_PROV         33809 non-null  object 
 4   COLEGIO_DIST         33809 non-null  object 
 5   COLEGIO_PAIS         33809 non-null  object 
 6   COLEGIO_ANIO_EGRESO  33809 non-null  int64  
 7   ESPECIALIDAD         33809 non-null  object 
 8   ANIO_POSTULA         33809 non-null  int64  
 9   CICLO_POSTULA        33809 non-null  int64  
 10  DOMICILIO_DEPA       33809 non-null  object 
 11  DOMICILIO_PROV       33809 non-null  object 
 12  DOMICILIO_DIST       33809 non-null  object 
 13  ANIO_NACIMIENTO      33809 non-null  int64  
 14  NACIMIENTO_PAIS      33809 non-null  object 
 15  NACIMIENTO_DEPA      33809 non-null 

In [10]:
# DEL ANÁLISIS REALIZADO POR DISTRIBUCIÓN GEOGRÁFICA ENTRE LOS DATOS DOMICILIO, COLEGIO Y NACIMIENTO, SE OBTUVO QUE EL DATO CON MAYOR PRECISIÓN Y CRITERIO ES EL DE COLEGIO, POR LO QUE SE UTILIZARÁ ESE DATOS PARA LA ZONIFICACIÓN DE PROVINCIAS.
# PARA IDENTIFICAR Y ANALIZAR DATOS DE ALUMNOS CON DOMICILIO, COLEGIO Y NACIMIENTO, SE AGRUPARÁN LOS DEPARTAMENTOS POR REGIONES, 'NO', 'NE', 'NC', 'SC', 'SE', 'SO', 'LC', 'I'
# EN LOS DATOS DE COLEGIO NO SE PRESENTAN REGISTROS QUE PERTENEZCAN A UN COLEGIO EN EL EXTRANJERO [(I)NTERNACIONAL]

# DELIMITACIÓN DE REGIONES GEOGRÁFICAMENTE SEGÚN CRITERIO DE CERCANÍA DE DEPARTAMENTOS
data=[
       ['TUMBES', 'PIURA', 'LAMBAYEQUE', 'CAJAMARCA'],     #NO
       ['AMAZONAS', 'LORETO', 'SAN MARTIN', 'UCAYALI'],    #NE
       ['LA LIBERTAD', 'ANCASH', 'HUANUCO', 'PASCO'],      #NC
       ['JUNIN', 'HUANCAVELICA', 'AYACUCHO', 'APURIMAC'],  #SC
       ['CUSCO', 'MADRE DE DIOS', 'PUNO'],                 #SE
       ['ICA', 'AREQUIPA', 'MOQUEGUA', 'TACNA'],           #SO
       ['CALLAO', 'LIMA']                                  #LC
     ]

def asignar_region(departamento_provincia):
  spl = departamento_provincia.split('-')
  if spl[0] in data[0]:
    return 'NO'
  elif spl[0] in data[1]:
    return 'NE'
  elif spl[0] in data[2]:
    return 'NC'
  elif spl[0] in data[3]:
    return 'SC'
  elif spl[0] in data[4]:
    return 'SE'
  elif spl[0] in data[5]:
    return 'SO'
  elif spl[0] in data[6]:
    if spl[1] in data[6]:
      return 'LC'
    else:
      return 'SC'
  else:
    return 'I'

df['COLEGIO_ZONA'] = df['COLEGIO_DEPA_PROV'].apply(asignar_region)

In [11]:
# PARA EL ANÁLISIS Y MODELO PREDICTIVO, SE CONSIDERA ÚNICAMENTE ALUMNOS INGRESANTES POR LA MODALIDAD INGRESO DIRECTO PARA REFORZAR EL ANÁLISIS CONCENTRADO EN INFORMACIÓN EXCLUSIVA DE LA CEPRE
# POR LO QUE SE CONSIDERA QUE LOS ALUMNOS QUE INGRESARON POR ORDINARIO, PREVIAMENTE NO INGRESARON POR CEPRE, ENTONCES:

# DEFINIR LAS CONDICIONES
condicion = (df['INGRESO']=='SI') & (df['MODO_INGRESO'] == 'ORDINARIO')

# REEMPLAZAR LOS VALORES DE 'INGRESO' BAJO ESTA CONDICIÓN
df.loc[condicion, 'INGRESO'] = 'NO'

dff = df

In [12]:
# CON LA DATA PREPARADA PARA APLICARLA EN EL MODELAMIENTO, SE EJECUTAN LOS SIGUIENTES PASOS:

# PASO 1: PREPROCESAR LOS DATOS Y ENTRENAR MODELO

# SELECCIONAR LAS CARACTERÍSTICAS RELEVANTES
features = ['SEXO', 'CALIF_FINAL', 'PREPARACION',
            'CICLO', 'EDAD', 'COLEGIO_ZONA']

# VARIABLE OBJETIVO
target = 'INGRESO'

# FILTRAR COLUMNAS NECESARIAS
X = dff[features]
y = dff[target]

# DEFINIR EL PREPROCESADOR
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['EDAD', 'CALIF_FINAL','PREPARACION']),
        ('cat', OneHotEncoder(handle_unknown='ignore'), ['SEXO', 'CICLO', 'COLEGIO_ZONA'])
    ])

# CREAR PIPELINE QUE INCLUYA EL PROCESADOR Y EL MODELO
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', LogisticRegression())
])

# DIVIDIR LOS DATOS EN CONJUNTO DE ENTRENAMIENTO Y PRUEBA
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ENTRENAR MODELO
model_pipeline.fit(X_train, y_train)

# PREDECIR CON DATOS TEST
y_pred = model_pipeline.predict(X_test)

# EVALUDAR EL RENDIMIENTO DEL MODELO
print("Exactitud:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))
y_pred_prob = model_pipeline.predict_proba(X_test)[:, 1]
print("AUC-ROC:", roc_auc_score(y_test, y_pred_prob))

Exactitud: 0.9279798876072168
              precision    recall  f1-score   support

          NO       0.95      0.97      0.96      6078
          SI       0.66      0.59      0.62       684

    accuracy                           0.93      6762
   macro avg       0.81      0.78      0.79      6762
weighted avg       0.92      0.93      0.93      6762

AUC-ROC: 0.9579129936555769


In [16]:
# Paso 2: PREDECIR CON VALORES NUEVOS

# NUEVOS DATOS PARA PREDECIR
df_nuevos = pd.DataFrame({
    'SEXO': ['FEMENINO', 'FEMENINO', 'FEMENINO', 'MASCULINO'],
    'EDAD': [21, 19, 18, 18],
    'CALIF_FINAL': [14, 13.4, 15, 16],
    'PREPARACION': [4, 2, 2, 2],
    'COLEGIO_ZONA': ['LC', 'SO', 'NC', 'LC'],
    'CICLO': ['2017-1', '2025-2', '2018-2', '2010-1']
})

# SE CONDIFICAN Y ESTANDARIZAN LOS NUEVOS DATOS
df_nuevos_encoded = preprocessor.transform(df_nuevos)
predicciones = model_pipeline.named_steps['model'].predict(df_nuevos_encoded)

# ALMACENAR LAS PREDICCIONES
df_nuevos['PREDICCIÓN'] = predicciones

df_nuevos

,SEXO,EDAD,CALIF_FINAL,PREPARACION,COLEGIO_ZONA,CICLO,PREDICCIÓN
0,FEMENINO,21,14.0,4,LC,2017-1,SI
1,FEMENINO,19,13.4,2,SO,2025-2,NO
2,FEMENINO,18,15.0,2,NC,2018-2,NO
3,MASCULINO,18,16.0,2,LC,2010-1,SI


In [17]:
# OBTENER EL MODELO DE REGRESIÓN LOGÍSTICA DEL PIPELINE
modelo_logistico = model_pipeline.named_steps['model']

# OBTENER COEFICIENTES DEL MODELO
coeficientes = modelo_logistico.coef_[0]

nombres_caracteristicas = preprocessor.get_feature_names_out()

# CREAR DF PARA VISUALIZAR LOS COEFICIENTES JUNTO A LOS NOMBRES DE LAS CARACTERÍSTICAS
coef_df = pd.DataFrame({'Característica': nombres_caracteristicas, 'Peso': coeficientes})

# ORDENAR EL VALOR ABSOLUTO DEL PESO
coef_df['Peso_absoluto'] = coef_df['Peso'].abs()
coef_df = coef_df.sort_values(by='Peso_absoluto', ascending=False)

coef_df

,Característica,Peso,Peso_absoluto
1,num__CALIF_FINAL,5.074844,5.074844
10,cat__CICLO_2019-1,-2.078215,2.078215
15,cat__CICLO_2022-1,1.750988,1.750988
9,cat__CICLO_2018-2,-1.513251,1.513251
13,cat__CICLO_2021-1,1.497065,1.497065
11,cat__CICLO_2019-2,-1.412508,1.412508
18,cat__CICLO_2023-2,1.294599,1.294599
17,cat__CICLO_2023-1,1.181808,1.181808
16,cat__CICLO_2022-2,1.110913,1.110913
14,cat__CICLO_2021-2,0.970334,0.970334


In [18]:
# DESCARGAR EL MODELO PREDICTIVO ENTRENADO PARA UTILIZARLO EN EL TABLERO DE CONTROL POWER BI

import pickle

# GUARDAR EN UN ARCHIVO .PKL
with open('modelo_pipeline.pkl', 'wb') as file:
    pickle.dump(model_pipeline, file)